# Make sure that Miniconda3 and MFA are isntall and working prior to running these cells
- command "mfa align --help" should print help

In [2]:
# macos.linux
# !mkdir ./aligned_pre
# windows
!mkdir .\aligned_pre


In [3]:
# copy wavs into aligned_pre
import os
import shutil

source_folder = r"../datasets/cmu_arctic/cmu_us_bdl_arctic/wav/"
destination_folder = r"./aligned_pre/"

# fetch all files
for file_name in os.listdir(source_folder):
    # construct full file path
    source = source_folder + file_name
    destination = destination_folder + "bdl_" + file_name
    # copy only files
    if os.path.isfile(source):
        if ".wav" in file_name:
            shutil.copy(source, destination)
            print('copied', file_name)

copied arctic_a0001.wav
copied arctic_a0002.wav
copied arctic_a0003.wav
copied arctic_a0004.wav
copied arctic_a0005.wav
copied arctic_a0006.wav
copied arctic_a0007.wav
copied arctic_a0008.wav
copied arctic_a0009.wav
copied arctic_a0010.wav
copied arctic_a0011.wav
copied arctic_a0012.wav
copied arctic_a0013.wav
copied arctic_a0014.wav
copied arctic_a0015.wav
copied arctic_a0016.wav
copied arctic_a0017.wav
copied arctic_a0018.wav
copied arctic_a0019.wav
copied arctic_a0020.wav
copied arctic_a0021.wav
copied arctic_a0022.wav
copied arctic_a0023.wav
copied arctic_a0024.wav
copied arctic_a0025.wav
copied arctic_a0026.wav
copied arctic_a0027.wav
copied arctic_a0028.wav
copied arctic_a0029.wav
copied arctic_a0030.wav
copied arctic_a0031.wav
copied arctic_a0032.wav
copied arctic_a0033.wav
copied arctic_a0034.wav
copied arctic_a0035.wav
copied arctic_a0036.wav
copied arctic_a0037.wav
copied arctic_a0038.wav
copied arctic_a0039.wav
copied arctic_a0040.wav
copied arctic_a0041.wav
copied arctic_a0

In [4]:
# create transcript files from metadata.csv
#lines = open('./ljs/LJSpeech-1.1/metadata.csv', 'r').readlines()
lines = open('../datasets/cmu_arctic/cmu_us_bdl_arctic/etc/txt.done.data', 'r').readlines()
from tqdm.auto import tqdm
for line in tqdm(lines):
    temp = line.split('"')
    transcript = temp[1]
    #print(transcript)
    fn = temp[0].split(' ')[1]
    #print(fn)
    ident = fn
    open(f'./aligned_pre/bdl_{ident}.txt', 'w').write(transcript)

# this is an example transcript for LJ001-0001.wav
#!cat ./wav/LJ001-0001.txt

100%|██████████| 1131/1131 [00:00<00:00, 1427.71it/s]


In [6]:
# download a pretrained english acoustic model, and english lexicon
# IF THE FILES ARE NOT PRESENT
!wget -q --show-progress https://github.com/MontrealCorpusTools/mfa-models/raw/main/acoustic/english.zip
!wget -q --show-progress -o ./librispeech-lexicon.txt http://www.openslr.org/resources/11/librispeech-lexicon.txt

Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
wget: unrecognised option `--show-progress'


In [16]:
# see: https://github.com/MontrealCorpusTools/Montreal-Forced-Aligner/pull/480
# ONLY RUN THIS IS MODIFIED FILE IS NOT PRESENT
import re
lexicon = open("./librispeech-lexicon.txt").readlines()
sp = re.compile("\s+")
with open("./modified_librispeech-lexicon.txt", "w") as f:
    for line in lexicon:
        word, *phonemes = sp.split(line.strip())
        phonemes = " ".join(phonemes)
        f.write(f"{word}\t{phonemes}\n")